In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import httpx as hx
import json

In [2]:
with open('sample_html.html','r') as b:
    sample_ = b.read()

In [14]:

def checksum_items(page:str,name:str):

    with open('../data/dumps/id_history.json','r') as buffer:
        parsed_ids = json.load(buffer)
    
    bs_data = BeautifulSoup(page, features='lxml')
    result = bs_data.find('div', {'class': 'items-items-kAJAg'}).findAll('div', {'data-marker': 'item'})

    new_items = [item['data-item-id'] for item in result if item['data-item-id'] not in parsed_ids[name]]
    
    parsed_ids[name] = parsed_ids[name] + new_items

    with open('../data/dumps/id_history.json','w') as buffer:
        json.dump(parsed_ids,buffer,indent=4)

    return new_items
    

In [15]:
checksum_items(sample_,'avito')

[]

In [1]:
import yaml

In [2]:
with open('../pipelines/config.yml') as buffer:
    zz = yaml.safe_load(buffer)

In [3]:
zz

{'ops': {'fetch_avito': {'config': {'fetch_pages': 10,
    'start_url': 'https://www.avito.ru/moskva/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?cd=1&f=ASgBAQICAkSSA8YQwMENuv03BEDmBxSMUsoIJIRZglmO3g4UApDeDhQC&s=104'}}}}